# ONNX to TorchScript Conversion

This notebook converts ONNX models to TorchScript format and validates the conversion.

In [ ]:
1+1

In [2]:
import torch
import torch.nn as nn
import onnx
import dill
from onnx2torch import convert
import os
import matplotlib.pyplot as plt  # Add this line

In [ ]:
# Check if running on Mac OS
is_mac = os.name == 'posix' and os.uname().sysname == 'Darwin'
print('posix' if os.name == 'posix' else 'not posix')
print('mac' if is_mac else 'not mac')


In [4]:
# Set up paths
is_mac = os.name == 'posix' and os.uname().sysname == 'Darwin'
rootPath = "~/Projects/AWI/NetExploration/" if is_mac else '/mnt/SliskiDrive/AWI/AWIBuffer/' # '/Volumes/Crucial X8/AWIBuffer/'
onnxPath = rootPath + "UNETR-nnUNetPlans_2d-DC_and_CE_loss-w-1-15-15-opset18.onnx"

In [ ]:
onnxPath

In [6]:
# Check if ONNX file exists
if not os.path.exists(os.path.expanduser(onnxPath)):
    raise FileNotFoundError(f"ONNX file not found at path: {onnxPath}")


In [7]:
onnxPath="UNETR-nnUNetPlans_2d-DC_and_CE_loss-w-1-15-15-opset18.onnx"

In [8]:
# Convert ONNX model to PyTorch
modelPerOnnx = convert(onnxPath)

In [ ]:
# Set up device
gpuDevice = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print(f"Using device: {gpuDevice}") 

In [10]:


modelPerOnnx = modelPerOnnx.to(gpuDevice)

## Test Model with Random Input

In [ ]:
# Create random test tensor
random_tensor = torch.randn(1, 5, 512, 512, device=gpuDevice, dtype=torch.float32)
print("Input tensor shape:", random_tensor.shape)

# Test model
modelPerOnnx.eval()
with torch.inference_mode():
    result = modelPerOnnx(random_tensor)

print("Output tensor shape:", result.shape)

## Test Model with HDF5 Input

In [ ]:
import platform

system = platform.system()
if "Darwin" in system:
    if os.path.isdir("/Volumes/Crucial X8"):
        dataDir = "/Volumes/Crucial X8/AWIBuffer"
    else:
        dataDir = "/Users/billb/Projects/AWI/NetExploration"
elif "Linux" in system:
    dataDir = "/mnt/SliskiDrive/AWI/AWIBuffer"
else:
    dataDir = None  # or some default path

angiogramH5Path = dataDir + "/AngiogramsDistilledUInt8List.h5"
angiogramH5Path

In [ ]:
import h5py

# Open the HDF5 file and print all dataset keys
with h5py.File(angiogramH5Path, 'r') as f:
    # Get all keys at root level
    keys = list(f.keys())
    print("Dataset keys in HDF5 file:")
    for key in keys:
        print(f"- {key}")


In [ ]:
# Load first angiogram from HDF5 file
import random
with h5py.File(angiogramH5Path, 'r') as f:
    # Get first key
    hdfKey = random.choice(keys)
    print(f"Loading dataset: {hdfKey}")
    # Load data into tensor
    agram = torch.from_numpy(f[hdfKey][:]).float()
    print(f"Loaded tensor shape: {agram.shape}")


In [ ]:
#Display the 30th frame of the angiogram
plt.imshow(agram[30], cmap='gray')
plt.colorbar()
plt.show()


In [ ]:
# Normalize angiogram by subtracting mean and dividing by standard deviation
xagram = (agram - agram.mean()) / agram.std()
print(f"Normalized tensor shape: {xagram.shape}")


In [ ]:
# Create input tensor with 5 consecutive frames centered around frame 30
start_idx = 28  # 30-2 to get 2 frames before
end_idx = 33    # 30+3 to get 2 frames after (exclusive)
z = xagram[start_idx:end_idx].unsqueeze(0)  # Add batch dimension
print(f"Input tensor shape: {z.shape}")


In [18]:
# Move model and input tensor to GPU device
gpuDevice = 'mps'
modelPerOnnx = modelPerOnnx.to(gpuDevice)
z = z.to(gpuDevice)


In [ ]:
y=modelPerOnnx(z)
y.shape


In [ ]:
# Apply softmax along dimension 1 (second dimension) which has size 3
y = torch.nn.functional.softmax(y, dim=1)
print(f"Output tensor shape after softmax: {y.shape}")


In [ ]:
# Display the 3rd channel (index 2) of the output
plt.imshow(y[0, 2].cpu().detach().numpy(), cmap='gray')
plt.colorbar()
plt.title('Output Channel 3')
plt.show()


In [ ]:
# Calculate number of valid frame groups (each group has 5 consecutive frames)
num_frames = xagram.shape[0]
num_groups = num_frames - 4  # Each group needs 5 frames

# Create tensor to hold all valid frame groups
z5 = torch.zeros((num_groups, 5, 512, 512))

# Fill z5 with overlapping groups of 5 consecutive frames
for i in range(num_groups):
    z5[i] = xagram[i:i+5]

print(f"Shape of tensor containing all valid 5-frame groups: {z5.shape}")


In [ ]:
# Feed z5 into the model and get the output
y5 = modelPerOnnx(z5.to('mps'))
y5.shape

In [ ]:
# Apply softmax along dimension 1 (second dimension) which has size 3
ys5 = torch.nn.functional.softmax(y5, dim=1)
print(f"Output tensor shape after softmax: {ys5.shape}")


In [ ]:
# Display the 3rd channel (index 2) of batch member 35
plt.imshow(ys5[35, 2].cpu().detach().numpy(), cmap='gray')
plt.colorbar()
plt.title('Output Channel 3 - Batch 35')
plt.show()


## Export Back to ONNX

In [28]:
# Export model back to ONNX
onnxOutputPath = onnxPath.replace(".onnx", "-torch-onnx.onnx")

# Move both model and input tensor to CPU for export
# model_for_export = modelPerOnnx.to(gpuDevice)
# input_for_export = z5.to(gpuDevice)

with torch.inference_mode():
    torch.onnx.export(modelPerOnnx.to('cpu'),
                     z.to('cpu'),
                     onnxOutputPath, 
                     export_params=True,
                     opset_version=18, 
                     do_constant_folding=True,
                     verbose=True,
                     input_names=['input'],
                     output_names=['output'], 
                     dynamic_axes={'input': {0: 'batch_size'}, 
                                 'output': {0: 'batch_size'}}, 
                     training=torch.onnx.TrainingMode.EVAL)


## Save and Load PyTorch Model

In [ ]:
# Save PyTorch model
torchModelPath = onnxPath.replace(".onnx", "-torch-onnx.pt")
torch.save(modelPerOnnx, torchModelPath)

# Load and verify
checkModel = torch.load(torchModelPath, weights_only=False)
checkModel.eval()
with torch.inference_mode():
    result = checkModel(random_tensor)

print("Verification output shape:", result.shape)

## TorchScript Conversion

In [ ]:
# Create example inputs with different batch sizes
example_input_1 = torch.randn(2, 5, 512, 512).to(gpuDevice)
example_input_2 = torch.randn(4, 5, 512, 512).to(gpuDevice)

# Create traced model
tracedModelperOnnx = torch.jit.trace(modelPerOnnx, example_input_1, check_trace=False)

# Save traced model
tracedModelPath = onnxPath.replace(".onnx", "-torchscript-traced-onnx.pt")
tracedModelperOnnx.save(tracedModelPath)

## Verify Traced Model

In [ ]:
# Load and test traced model
tracedModel = torch.jit.load(tracedModelPath)
tracedModel = tracedModel.to(gpuDevice)

tracedModel.eval()
with torch.inference_mode():
    result = tracedModel(example_input_2)

print("Traced model output shape:", result.shape)

# Test with different batch size
result = tracedModelperOnnx(example_input_2)
print("Different batch size output shape:", result.shape)